In [93]:
import streamlit as st
import pandas as pd

from googleapiclient.discovery import build
from google.oauth2 import service_account
from io import BytesIO
from googleapiclient.http import MediaIoBaseDownload

from datetime import datetime
import pytz

import plotly.express as px

SCOPES = ['https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = 'service_account.json'
PARENT_FOLDER_ID = '1vIKk9xgUn1JGY2MFPdB6W3tEfU8oVptP'

def authenticate():
    '''
    Autentikasi akun untuk akses ke gdrive
    '''
    creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    return creds

def list_files():
    creds = authenticate()
    service = build('drive', 'v3', credentials=creds)
    
    results = service.files().list(
        q=f"'{PARENT_FOLDER_ID}' in parents",
        spaces='drive',
        fields='nextPageToken, files(id, name, modifiedTime)',
        pageSize=10
    ).execute()
    
    items = results.get('files', [])
    
    file_list = []
    if items:
        file_list = [(item['name'], item['id'], item['modifiedTime']) for item in items]
    return file_list

def read_file_from_drive(file_id):
    '''
    Read Excel file from Google Drive
    '''
    creds = authenticate()
    service = build('drive', 'v3', credentials=creds)
    
    # Stream the file content
    request = service.files().get_media(fileId=file_id)
    file_data = BytesIO()
    downloader = MediaIoBaseDownload(file_data, request)
    done = False
    while not done:
        _, done = downloader.next_chunk()
    
    file_data.seek(0)
    return file_data  # Return the raw file data

def read_file_from_drive(file_id):
    '''
    Read Excel file from Google Drive
    '''
    creds = authenticate()
    service = build('drive', 'v3', credentials=creds)
    
    # Stream the file content
    request = service.files().get_media(fileId=file_id)
    file_data = BytesIO()
    downloader = MediaIoBaseDownload(file_data, request)
    done = False
    while not done:
        _, done = downloader.next_chunk()
    
    file_data.seek(0)
    return file_data  # Return the raw file data

def processing_excel(file_data, sheet_name):
    '''
    Process Excel file to clean and prepare the data
    '''
    
    excel_ptr = pd.read_excel(file_data, sheet_name, header=None)
    temp_df = excel_ptr.ffill()
    
    listof_ver = temp_df[temp_df[1].str.contains('PTR Ver', na=False)][1].unique().tolist()
    
    # Get the rows which will become header
    value_to_skip = 'Features'
    max_rows_to_scan = 10

    header_index = excel_ptr.head(max_rows_to_scan).apply(lambda row: row.astype(str).str.contains(value_to_skip).any(), axis=1).idxmax()

    if pd.isna(header_index):  # If 'Features' wasn't found in the scanned rows
        print("Header 'Features' not found in the first", max_rows_to_scan, "rows.")
    else:
        new_header = excel_ptr.iloc[header_index].values
        excel_ptr = excel_ptr.iloc[header_index+1:].copy()
        excel_ptr.columns = new_header
        
    excel_ptr = excel_ptr.iloc[:, 1:]
    excel_ptr.reset_index(drop=True, inplace=True)
    
    # Convert column OS Version types
    if 'OS Version' not in excel_ptr.columns:
        st.warning("The selected sheet does not have an 'OS Version' column. Skipping this part of processing.")
    else:
        excel_ptr['OS Version'] = excel_ptr['OS Version'].astype(str)
    
    # drop column Number
    excel_ptr.drop(columns='No', inplace=True, errors='ignore')
    
    #Rename the column
    excel_ptr.rename(columns={
        'Sub Fitur': 'Sub-features',
        'Rekening Sumber\n[Jika ada]': 'Rekening Sumber',
        'Data yang Digunakan\n[Jika ada]': 'Data yang digunakan',
        'FT\n[Jika Ada]': 'FT',
        'Tanggal Eksekusi\n[harus diisi]': 'Tanggal Eksekusi',
        'Tanggal Passed\n[harus diisi]': 'Tanggal Passed'
    }, inplace=True)
    
    # Because of merged and centered, need to use ffill to duplicate values before current rows
    excel_ptr[['Features', 'Sub-features', 'Expected Condition']] = excel_ptr[['Features', 'Sub-features', 'Expected Condition']].apply(lambda x: x.ffill())
    
    return excel_ptr, listof_ver

def status_progress(processed_excel, version):
    """
    Calculate the progress based on the processed Excel data.
    """
    statusBased_OS = [(x, y) for x, y in zip(processed_excel['OS'].values, processed_excel['Status '+ version].values)]
    passed_prog = sum(1 for item in statusBased_OS if item == ('Android', 'Passed'))
    failed_prog = sum(1 for item in statusBased_OS if item == ('Android', 'Failed'))
    unknown_prog = sum(1 for item in statusBased_OS if item == ('Android', 'N/A'))
    inprog_prog = sum(1 for item in statusBased_OS if item == ('Android', 'In Progress'))
    
    return passed_prog, failed_prog, unknown_prog, inprog_prog


In [94]:
file_list = list_files()

In [95]:
[(file_id, modified_time) for name, file_id, modified_time in file_list if name == 'PTR_Rilis_D.xlsx'][0][0]

'1JzRCBOTHgisl-ckN58w9TA_t4bUgMusv'

In [96]:
selected_file_data = [(file_id, modified_time) for name, file_id, modified_time in file_list if name == 'PTR_Rilis_D.xlsx'][0][0]
selected_file_id = selected_file_data

file_data = read_file_from_drive(selected_file_id)

In [97]:
df, version = processing_excel(file_data, 'Prototype - Tiket Prod Issue PO')
df.head(3)

,Features,Sub-features,Expected Condition,OS,OS Version,Tipe Device HP,Telko Provider HP,Rekening Sumber,Data yang digunakan,PO,FT,Status PTR Ver Android 1.0.3 (272) iOS 1.0.3 (266),Status PTR Ver Android 1.0.3 (272) iOS 1.0.3 (267),Link Report Test,Description Issue dan Evidence,Tanggal Eksekusi PTR Ver Android 1.0.3 (272) iOS 1.0.3 (266),Tanggal Passed PTR Ver Android 1.0.3 (272) iOS 1.0.3 (266),Tanggal Eksekusi PTR Ver Android 1.0.3 (272) iOS 1.0.3 (267),Tanggal Passed PTR Ver Android 1.0.3 (272) iOS 1.0.3 (267),Komentar
0,PS-495 : RELEASE A - IOS - Tidak bisa klik ban...,Donasi,Byond dapat menampilkan halaman list LAZ Donasi,iOS,16.3,iPhone XS,Telkomsel,NaN,NaN,"Iin, Dhimas",NaN,Passed,Not Started,NaN,evidence :\n\nhttps://drive.google.com/drive/f...,2024-11-05 00:00:00,2024-11-05 00:00:00,NaN,NaN,Scenario :\n1. Login Byond dengan iOS\n2. Klik...
1,PS-833 : IOS - TRF TERJADWAL SEKALI GAGAL,Donasi,Berhasil melakukan trf terjadwal dalam jangka ...,Android,13,Galaxy S20+,Telkomsel,NaN,NaN,NaN,NaN,Passed,Not Started,https://bsicenter-my.sharepoint.com/:v:/g/pers...,NaN,2024-11-05 00:00:00,2024-11-05 00:00:00,NaN,NaN,NaN
2,PS-833 : IOS - TRF TERJADWAL SEKALI GAGAL,Donasi,Berhasil melakukan trf terjadwal dalam jangka ...,iOS,17.6.1,iPhone 12,Telkomsel,NaN,NaN,NaN,NaN,Passed,Not Started,https://bsicenter-my.sharepoint.com/:v:/g/pers...,NaN,2024-11-05 00:00:00,2024-11-05 00:00:00,NaN,NaN,NaN


In [99]:
df.Features[0]

'PS-495 : RELEASE A - IOS - Tidak bisa klik banner donasi - IIN'

In [82]:
version_chosen = version[0]
version_chosen

'PTR Ver Android 1.0.3 (272) iOS 1.0.3 (266)'

In [83]:
df = df.drop(columns=[col for col in df.columns if (col.startswith('Status ') or col.startswith('Tanggal Eksekusi ') or col.startswith('Tanggal Passed ')) \
    and col not in ['Status ' + version_chosen, 'Tanggal Eksekusi ' + version_chosen, 'Tanggal Passed ' + version_chosen]])

In [84]:
df.head(3)

,Features,Sub-features,Expected Condition,OS,OS Version,Tipe Device HP,Telko Provider HP,Rekening Sumber,Data yang digunakan,PO,FT,Status PTR Ver Android 1.0.3 (272) iOS 1.0.3 (266),Link Report Test,Description Issue dan Evidence,Tanggal Eksekusi PTR Ver Android 1.0.3 (272) iOS 1.0.3 (266),Tanggal Passed PTR Ver Android 1.0.3 (272) iOS 1.0.3 (266),Komentar
0,PS-495 : RELEASE A - IOS - Tidak bisa klik ban...,Donasi,Byond dapat menampilkan halaman list LAZ Donasi,iOS,16.3,iPhone XS,Telkomsel,NaN,NaN,"Iin, Dhimas",NaN,Passed,NaN,evidence :\n\nhttps://drive.google.com/drive/f...,2024-11-05 00:00:00,2024-11-05 00:00:00,Scenario :\n1. Login Byond dengan iOS\n2. Klik...
1,PS-833 : IOS - TRF TERJADWAL SEKALI GAGAL,Donasi,Berhasil melakukan trf terjadwal dalam jangka ...,Android,13,Galaxy S20+,Telkomsel,NaN,NaN,NaN,NaN,Passed,https://bsicenter-my.sharepoint.com/:v:/g/pers...,NaN,2024-11-05 00:00:00,2024-11-05 00:00:00,NaN
2,PS-833 : IOS - TRF TERJADWAL SEKALI GAGAL,Donasi,Berhasil melakukan trf terjadwal dalam jangka ...,iOS,17.6.1,iPhone 12,Telkomsel,NaN,NaN,NaN,NaN,Passed,https://bsicenter-my.sharepoint.com/:v:/g/pers...,NaN,2024-11-05 00:00:00,2024-11-05 00:00:00,NaN


In [85]:
df_android = df[df['OS'] == 'Android']

In [86]:
import numpy as np
df_android['Status '+ version_chosen] = df_android['Status ' + version_chosen].replace(np.nan, 'Not Started')

C:\Users\nahar\AppData\Local\Temp\ipykernel_14960\1312768736.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [87]:
df_android['Status ' + version_chosen].value_counts().apply(lambda x:x/len(df_android)*100)

Status PTR Ver Android 1.0.3 (272) iOS 1.0.3 (266)
Passed         38.888889
Not Started    38.888889
Failed         16.666667
In Progress     5.555556
Name: count, dtype: float64

In [88]:
progress_visual(df, version_chosen)

NameError: name 'progress_visual' is not defined

In [18]:
def progress_status(df, version):
    df_android = df[df['OS'] == 'Android'].copy()
    df_ios = df[df['OS'] == 'iOS'].copy()

    def process_data(data, version):
        data['Status ' + version] = data['Status ' + version].replace(np.nan, 'N/A')
        return data['Status ' + version].value_counts(normalize=True) * 100

    android_result = process_data(df_android, version)
    ios_result = process_data(df_ios, version)
    
    df_plot = pd.concat([android_result.rename("Android"), ios_result.rename('iOS')], axis=1).reset_index()
    df_plot.rename(columns={'Status '+version : 'Status'}, inplace=True)
    
    df_plot = df_plot.melt(id_vars='Status', var_name='Platform', value_name='Percentage')
    df_plot['Percentage'] = df_plot['Percentage'].apply(lambda row: round(row, 2))

    return df_plot

def progress_plot(df_plot, version):
    progress_bar = px.bar(
        df_plot,
        x="Status",
        y="Percentage",
        color="Platform",
        barmode="group",
        title=f"Status Distribution by Platform (Version {version})",
        text="Percentage",
        color_discrete_map={"Android": "#FFA500", "iOS": "#1E90FF"}  # Custom colors
    )

    # Customize the traces for better readability
    progress_bar.update_traces(
        texttemplate='%{text:.2f}%',
        textposition='outside',
        marker=dict(line=dict(width=1.5, color="black"))  # Add a border to bars
    )

    # Update layout for a cleaner and professional look
    progress_bar.update_layout(
        title=dict(
            text=f"<b>Status Distribution by Platform (Version {version})</b>",
            font=dict(size=20, family="Arial, sans-serif"),
            x=0.5  # Center the title
        ),
        xaxis=dict(
            title="<b>Status</b>",
            tickangle=-45,  # Tilt x-axis labels for readability
            tickfont=dict(size=12, family="Arial, sans-serif"),
        ),
        yaxis=dict(
            title="<b>Percentage (%)</b>",
            tickfont=dict(size=12, family="Arial, sans-serif"),
        ),
        legend=dict(
            title="<b>Platform</b>",
            font=dict(size=12, family="Arial, sans-serif"),
            bgcolor="#1E1E1E",  # Light gray background for legend
            bordercolor="black",
            borderwidth=1,
        ),
        margin=dict(l=50, r=50, t=80, b=50),  # Adjust margins for spacing
        plot_bgcolor="#1E1E1E",  # Light gray background
        paper_bgcolor="#1E1E1E",  # Light gray background
        width=1000,
        height=600
    )
    
    return progress_bar

In [19]:
df_plot = progress_status(df=df, version=version_chosen)
df_plot

,Status,Platform,Percentage
0,Passed,Android,38.89
1,N/A,Android,22.22
2,Failed,Android,16.67
3,Not Started,Android,16.67
4,In Progress,Android,5.56
5,Passed,iOS,52.63
6,N/A,iOS,5.26
7,Failed,iOS,15.79
8,Not Started,iOS,26.32
9,In Progress,iOS,NaN


In [20]:
progress_plot(df_plot=df_plot, version=version_chosen)

In [48]:
def truncate_and_wrap(text, max_words=5):
    words = text.split()
    if len(words) > max_words:
        first_line = " ".join(words[:max_words])
        second_line = " ".join(words[max_words:max_words*2])
        truncated_text = f"{first_line}\n{second_line}"
    else:
        truncated_text = " ".join(words)
    return truncated_text

def normalize_text(text):
    if isinstance(text, str):
        return text.strip().lower()
    else:
        return str(text).lower()  # Convert to string if it's not already, then normalize

In [53]:
import textwrap

def wrap_text(text, width=50):
    return '\n'.join(textwrap.wrap(text, width))

In [ ]:
node_indices[]

KeyError: 2

In [133]:
nodes = list(set(
    df['Features'].tolist() +
    df['Sub-features'].tolist() +
    df['OS'].tolist() +
    df['OS Version'].tolist() +
    df['Tipe Device HP'].tolist() +
    df["Status "+ version_chosen].tolist()
))

def wrap_long_name(name, width=50):
    return '<br>'.join(textwrap.wrap(str(name), width))

# Create short labels for Sankey plot without modifying the original df
node_indices = {
    node: (str(node)[:20] + "...") if isinstance(node, str) and len(str(node)) > 20 else str(node)
    for node in nodes
}

# Create a list of shortened node labels for the Sankey plot
short_nodes = [node_indices[node] for node in nodes]

long_nodes = [wrap_long_name(node) for node in nodes]

# Create flows (source → target)
sources = []
targets = []
values = []

# Generate sources and targets for the Sankey diagram
for _, row in df.iterrows():
    feature = row["Features"]
    sub_feature = row["Sub-features"]
    status = row["Status "+ version_chosen]
    os_type = row["OS"]

    # Feature -> Status -> OS if Status is "Passed"
    if status == "Passed":
        sources.append(nodes.index(feature))  # Feature -> Status
        targets.append(nodes.index(status))
        values.append(1)

        sources.append(nodes.index(status))  # Status -> OS
        targets.append(nodes.index(os_type))
        values.append(1)

    # Feature -> Sub-feature -> Status -> OS if Status is "Failed"
    elif status == "Failed" or status == "N/A" or status == "In Progress" or status == "Not Started":
        sources.append(nodes.index(feature))  # Feature -> Sub-feature
        targets.append(nodes.index(sub_feature))
        values.append(1)

        sources.append(nodes.index(sub_feature))  # Sub-feature -> Status
        targets.append(nodes.index(status))
        values.append(1)

        sources.append(nodes.index(status))  # Status -> OS
        targets.append(nodes.index(os_type))
        values.append(1)
        
    # Calculate incoming and outgoing flows
    incoming_flows = {node: 0 for node in nodes}
    outgoing_flows = {node: 0 for node in nodes}

    for source, target in zip(sources, targets):
        outgoing_flows[nodes[source]] += 1  # Increase the outgoing flow count for the source node
        incoming_flows[nodes[target]] += 1  # Increase the incoming flow count for the target node

    # Prepare customdata with both incoming and outgoing flows
    customdata = [
        f"{long_name} <br>Incoming: {incoming_flows[node]} <br>Outgoing: {outgoing_flows[node]}"
        for node, long_name in zip(nodes, long_nodes)
    ]
        
    # Assign a default value of 1 for each connection
    values = [1] * len(sources)

    # Use a qualitative color scale for high contrast (e.g., Plotly's 'Dark24')
    color_palette = px.colors.qualitative.Light24 # Replace with another palette if needed
    num_colors = len(color_palette)

    opacity = 1  # Example opacity value (0.0 - 1.0)
    node_colors = {}
    for i, feature in enumerate(df["Features"].unique()):
        hex_color = color_palette[i % num_colors]  # Get the hex color
        rgba_color = to_rgba(hex_color, alpha=opacity)  # Convert to RGBA
        node_colors[feature] = f"rgba({int(rgba_color[0]*255)}, {int(rgba_color[1]*255)}, {int(rgba_color[2]*255)}, {rgba_color[3]})"

    # Propagate feature colors to sub-features
    for feature in df["Features"].unique():
        feature_color = node_colors[feature]
        for sub_feature in df[df["Features"] == feature]["Sub-features"].unique():
            node_colors[sub_feature] = feature_color

    # Overwrite colors for 'Passed' and 'Failed'
    node_colors["Passed"] = "rgba(144, 238, 144, 0.8)"  # Soft green
    node_colors["Failed"] = "rgba(205, 92, 92, 0.8)"    # Soft red brick

    node_colors["Android"] = "rgba(64, 224, 208, 0.8)"  # Green turquoise
    node_colors["iOS"] = "rgba(70, 130, 180, 0.8)"      # Steel blue

    # Set default color for unassigned nodes
    default_color = "rgba(200, 200, 200, 0.8)"
    node_color_list = [node_colors.get(node, default_color) for node in nodes]

    # Assign link colors based on source node color with transparency
    link_colors = []
    for source, target in zip(sources, targets):
        source_color = node_colors.get(nodes[source], "rgba(192, 192, 192, 0.3)")  # Default gray if missing
        rgba_values = source_color.strip("rgba()").split(",")
        if len(rgba_values) == 4:
            r, g, b, _ = map(float, rgba_values[:4])
            link_colors.append(f"rgba({int(r)}, {int(g)}, {int(b)}, 0.3)")  # Reduce opacity for the links
        else:
            link_colors.append("rgba(192, 192, 192, 0.3)")  # Default gray


# Plot Sankey Diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=short_nodes,  # Use short labels here
        color=node_color_list,  # Optionally set a default color
        customdata=customdata,
        hovertemplate="%{customdata}<extra></extra>"
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color=link_colors
    )
)])

# Update layout and show
fig.update_layout(
    font_size=12,
    width=1000,  # Increase width for a wider graph
    height=700,   # Increase height for a taller graph
    font=dict(size=14, color='white'),
    plot_bgcolor='#1E1E1E',
    paper_bgcolor='#1E1E1E',
    margin=dict(l=20, r=20, t=20, b=20)
)

fig.show()


In [134]:
link_colors

['rgba(253, 50, 22, 0.3)',
 'rgba(144, 238, 144, 0.3)',
 'rgba(0, 254, 53, 0.3)',
 'rgba(144, 238, 144, 0.3)',
 'rgba(0, 254, 53, 0.3)',
 'rgba(144, 238, 144, 0.3)',
 'rgba(106, 118, 252, 0.3)',
 'rgba(214, 38, 255, 0.3)',
 'rgba(205, 92, 92, 0.3)',
 'rgba(106, 118, 252, 0.3)',
 'rgba(214, 38, 255, 0.3)',
 'rgba(205, 92, 92, 0.3)',
 'rgba(254, 212, 196, 0.3)',
 'rgba(144, 238, 144, 0.3)',
 'rgba(254, 212, 196, 0.3)',
 'rgba(144, 238, 144, 0.3)',
 'rgba(254, 0, 206, 0.3)',
 'rgba(254, 0, 206, 0.3)',
 'rgba(205, 92, 92, 0.3)',
 'rgba(254, 0, 206, 0.3)',
 'rgba(254, 0, 206, 0.3)',
 'rgba(205, 92, 92, 0.3)',
 'rgba(13, 249, 255, 0.3)',
 'rgba(144, 238, 144, 0.3)',
 'rgba(13, 249, 255, 0.3)',
 'rgba(144, 238, 144, 0.3)',
 'rgba(246, 249, 38, 0.3)',
 'rgba(255, 150, 22, 0.3)',
 'rgba(192, 192, 192, 0.3)',
 'rgba(255, 150, 22, 0.3)',
 'rgba(144, 238, 144, 0.3)',
 'rgba(71, 155, 85, 0.3)',
 'rgba(144, 238, 144, 0.3)',
 'rgba(71, 155, 85, 0.3)',
 'rgba(144, 238, 144, 0.3)',
 'rgba(238, 166, 251

In [125]:
long_nodes[1]

'18.01'

In [117]:
df.Features[0]

'PS-495 : RELEASE A - IOS - Tidak bisa klik banner donasi - IIN'